In [45]:
#데이터 가져오기
df = pd.read_csv("raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("raw_data/cgoongoo1.csv", encoding ='utf-8')

In [46]:
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district'], axis=1, inplace =True)

In [47]:
df_merged.columns

Index(['year', 'code', 'sales_total', 's_store_no_of_store',
       's_store_no_of_opening', 's_store_no_of_closing', 's_work_female_40s',
       's_work_female_50s', 's_work_female_60s', 'b_facil_total',
       's_float_male', 's_float_female', 'b_apt_avg_price',
       'b_income_avg_monthly_inc', 's_work_female', 's_work_male',
       'sales_weekday', 'sales_female', 'sales_2030s', 'sales_06_11',
       'sales_11_14', 'sales_14_17', 'sales_17_21', 'sales_21_24', 'cgoongoo'],
      dtype='object')

In [48]:
df_cgoongu_gb=df_merged.groupby(['year','cgoongoo', 'code']).agg({
    "sales_total":"sum",
    "s_store_no_of_store":"sum", #이걸 mean으로 해야할지 고민이 필요
    "s_store_no_of_opening":"sum",
    "s_store_no_of_closing":"sum",
    "sales_weekday":"sum",
    "sales_female":"sum",
    "sales_2030s":"sum",
    "sales_06_11":"sum",
    "sales_11_14":"sum",
    "sales_14_17":"sum",
    "sales_17_21":"sum",
    "sales_21_24":"sum",
    "s_work_female_40s":"mean",
    "s_work_female_50s":"mean",
    "s_work_female_60s":"mean",
    "b_facil_total":"mean",
    "s_float_male":"mean",
    "s_float_female":"mean",
    'b_apt_avg_price':'mean',
    'b_income_avg_monthly_inc':'mean',
    's_work_female':'mean',
    's_work_male':'mean',
    'b_income_avg_monthly_inc':'mean'
    
})
df_cgoongu_gb.tail(3)

sales_total  s_store_no_of_store  \
year cgoongoo code                                          
2019 11740    CS300015  5.912349e+09                   49   
              CS300016  5.516874e+09                  496   
              CS300017  2.322398e+09                   95   

                        s_store_no_of_opening  s_store_no_of_closing  \
year cgoongoo code                                                     
2019 11740    CS300015                      1                      1   
              CS300016                      8                      7   
              CS300017                      0                      1   

                        sales_weekday  sales_female  sales_2030s  sales_06_11  \
year cgoongoo code                                                              
2019 11740    CS300015       4.290286      3.237088     0.762611     0.080720   
              CS300016      25.429959      9.793725     6.189211     7.338632   
              CS300017       5.827562      4.553235     3.578226     0.315342   

                        sales_11_14  sales_14_17  ...  s_work_female_40s  \
year cgoongoo code                                ...                      
2019 11740    CS300015     2.089863     2.577843  ...             64.250   
              CS300016     7.644050    10.612261  ...             50.375   
              CS300017     3.444519     3.303651  ...             51.000   

                        s_work_female_50s  s_work_female_60s  b_facil_total  \
year cgoongoo code                                                            
2019 11740    CS300015          64.083333          34.333333     112.500000   
              CS300016          51.750000          28.187500     108.156250   
              CS300017          52.722222          33.666667     118.888889   

                         s_float_male  s_float_female  b_apt_avg_price  \
year cgoongoo code                                                       
2019 11740    CS300015  194778.250000   189097.833333     2.004435e+08   
              CS300016  158058.328125   154031.750000     1.826516e+08   
              CS300017  167489.388889   165458.055556     1.682300e+08   

                        b_income_avg_monthly_inc  s_work_female  s_work_male  
year cgoongoo code                                                            
2019 11740    CS300015              3.384987e+06     253.500000   274.166667  
              CS300016              3.370583e+06     205.921875   221.187500  
              CS300017              3.212065e+06     216.277778   193.944444  

[3 rows x 22 columns]

In [49]:
# # df 저장
# #index에 names=['year', 'cgoongoo', 'code']가 들어있다
# df_cgoongu_gb.to_csv('raw_data/df_cgoongu_gb.csv', index=True, encoding='utf-8')

In [50]:
df_cgoongu_gb.reset_index(inplace=True)

In [51]:
#로버스트 스케일링
from sklearn.preprocessing import RobustScaler
scale_columns = [i for i in df_cgoongu_gb.columns if i not in ['year', 'cgoongu', 'code',]]

rb = RobustScaler()
rb.fit(df_cgoongu_gb[scale_columns])
df_robust_scaled = rb.transform(df_cgoongu_gb[scale_columns])
df_robust_scaled = pd.DataFrame(df_robust_scaled, columns=scale_columns)
df_robust_scaled.columns
model= sm.OLS.from_formula("sales_total ~ C(cgoongoo) +"+" + ".join(scale_columns[2:]), data=df_robust_scaled)
result =  model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            sales_total   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     299.7
Date:                Tue, 24 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:23:24   Log-Likelihood:                -7585.1
No. Observations:                5591   AIC:                         1.526e+04
Df Residuals:                    5545   BIC:                         1.557e+04
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               